In [1]:
import numpy as np
import torch
import gc
from tqdm import tqdm, trange

import pickle
from time import time

from qaoa.quantum_model import QuantumModel

In [2]:
from functools import lru_cache

def save(filename, data):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

def load(filename,):
    with open(filename, 'rb') as f:
        return pickle.load(f)

@lru_cache
def repeated_tensor(base, exp):
    result = base

    if exp > 1:
        result = repeated_tensor(torch.kron(base, base), exp//2)
        if exp % 2 == 1:
            result = torch.kron(result, base)

    return result

def coupling_eig(coupling_tensor):
    eigvals = None

    pauli_diags = [torch.tensor([1, 1], dtype=torch.complex128), torch.tensor([1, -1], dtype=torch.complex128)]
    pauli_types = set()

    for ndx in coupling_tensor.keys():
        coeff = coupling_tensor[ndx]

        term = None

        for i in ndx:
            pauli_types.add(i)
            term = torch.kron(term, pauli_diags[min(i, 1)]) if term is not None else pauli_diags[min(i, 1)]
        term = term * coeff
        eigvals = eigvals + term if eigvals is not None else term

    if (1 in pauli_types and 2 in pauli_types) or (2 in pauli_types and 3 in pauli_types) or (3 in pauli_types and 1 in pauli_types):
        raise NotImplementedError("NotImplemented for more than one pauli type")

    if 1 in pauli_types: # X
        eigvecs = torch.tensor([[1, 1], [1, -1]], dtype=torch.complex128) / np.sqrt(2)
    if 2 in pauli_types: # Y
        eigvecs = torch.tensor([[1, 1], [1j, -1j]], dtype=torch.complex128) / np.sqrt(2)
    else: # I and/or Z
        eigvecs = torch.tensor([[1, 0], [0, 1]], dtype=torch.complex128)

    eigvecs = repeated_tensor(eigvecs, coupling_tensor.n)

    return eigvals, eigvecs

def parameter_eigs_exp(parameters, eigvals, eigvecs):

    N = len(eigvals)

    output_shape = parameters.shape + torch.Size([N, N])

    parameters = parameters.flatten()
    
    # (p), (N) -> (p, N)
    eig_exps = torch.exp(torch.einsum('p,N->pN', parameters, eigvals))

    # recombine (p, N), (N, N) -> (p, N, N)
    eig_exps = torch.einsum('pi,ij->pij', eig_exps, eigvecs.adjoint())
    eig_exps = torch.einsum('ij,pjk->pik', eigvecs, eig_exps)

    return eig_exps.reshape(output_shape)

In [3]:
class QAOA_torch:
    def __init__(self, hamiltonian, device="cpu"):
        self.hamiltonian = hamiltonian

        self.n = hamiltonian.num_qubits
        self.N = 2 ** self.n

        self.driver_coupling = self.hamiltonian.driver_coupling
        self.mixer_coupling = self.hamiltonian.mixer_coupling

        self.driver = torch.from_numpy(self.hamiltonian.driver.to_matrix()).to(device)
        self.mixer = torch.from_numpy(self.hamiltonian.mixer.to_matrix()).to(device)
        self.target = torch.from_numpy(self.hamiltonian.target.to_matrix()).to(device)
        
    def energy(self, gammas, betas):
        
        assert gammas.shape == betas.shape
        param_shape = gammas.shape

        batches, p = param_shape
        
        # (batches, N)
        psi = torch.ones(batches, self.N, dtype=torch.complex128) / np.sqrt(self.N)

        driver_eigvals, driver_eigvecs = coupling_eig(self.driver_coupling)
        mixer_eigvals, mixer_eigvecs = coupling_eig(self.mixer_coupling)

        exp_driver = parameter_eigs_exp(gammas.T * -1j, driver_eigvals, driver_eigvecs)
        exp_mixer = parameter_eigs_exp(betas.T * -1j, mixer_eigvals, mixer_eigvecs)
        
        for i in range(p):
            psi = torch.einsum('bij,bj->bi', exp_driver[i], psi)
            psi = torch.einsum('bij,bj->bi', exp_mixer[i], psi)
        
        del exp_driver
        del exp_mixer
        gc.collect()

        result = torch.einsum('ij,bj->bi', self.target, psi)
        result = torch.einsum('bi,bi->b', psi.conj(), result)

        return result.real
    
    def estimate_chunk_size(self, p):
        gb_used = 20
        
        psi_size = self.N
        ham_size = p * 2 * self.N * self.N
        coord_size = (2 * p) * (self.N ** (2 * p))

        chunk_size = (gb_used/3 * 1e9 - (16 * coord_size)) / (32 * (psi_size + ham_size))

        return max(chunk_size, 10)

    def energy_landscape(self, res, p, chunk_size = None):
        s = torch.linspace(0, 2 * np.pi, res + 1)[:-1]
        coords = torch.meshgrid(*[s] * (2 * p))
        coords = torch.stack([coord.flatten() for coord in coords]).T
        gammas, betas = torch.hsplit(coords, 2)

        if chunk_size is None:
            chunk_size = self.estimate_chunk_size(p)

        num_chunks = int(max((res ** (2 * p)) / chunk_size, 1))
        
        gamma_chunks = torch.chunk(gammas, num_chunks)
        beta_chunks = torch.chunk(betas, num_chunks)

        # clear massive memory used
        del s
        del coords
        del gammas
        del betas
        gc.collect()

        energies = []
        for i in trange(num_chunks):
            energies.append(self.energy(gamma_chunks[i], beta_chunks[i]))

        energies = torch.cat(energies)
        energies = energies.reshape([res] * (2 * p))
        return energies
    
    

In [4]:
"""
Schwinger Model
"""

select = {'N': 4, #qubits
          'g' : 1,  #coupling
          'm' : 1,  #bare mass
          'a' : 1, #lattice spacing
          'theta' : 0, #topological term
          'mixer_type' : 'Y', # type of mixer {'X', 'Y', 'XY'}
         }

model = QuantumModel('schwinger', 'Standard')


ham = model.make(select)
schwinger_target = ham.target

torch_qaoa = QAOA_torch(ham)

ham

driver:
4.0 * IIII
+ 2.0 * IIZZ
+ 1.0 * IZIZ
+ 1.0 * IZZI
+ 4.0 * IIIZ
- 1.0 * IIZI
+ 3.0 * IZII
- 2.0 * ZIII
mixer:
1.0 * IIIY
+ 1.0 * IIYI
+ 1.0 * IYII
+ 1.0 * YIII
target:
1.0 * IIII
+ 0.5 * IIZZ
+ 0.25 * IZIZ
+ 0.25 * IZZI
+ 1.0 * IIIZ
- 0.25 * IIZI
+ 0.75 * IZII
- 0.5 * ZIII
+ 0.25 * IIXX
+ 0.25 * IIYY
+ 0.25 * IXXI
+ 0.25 * IYYI
+ 0.25 * XXII
+ 0.25 * YYII

In [6]:
p = 1
res = 32
torch_qaoa = QAOA_torch(ham)
energy_scape = torch_qaoa.energy_landscape(res, p)
save(f"data/{p}p_schwinger_{res}", energy_scape)

100%|██████████| 1/1 [00:00<00:00, 17.11it/s]


In [6]:
energy_scape

tensor([[1.7500, 1.4039, 1.1679,  ..., 2.8873, 2.5821, 2.1693],
        [0.8227, 1.0090, 1.1266,  ..., 0.4866, 0.5095, 0.6336],
        [0.5581, 0.6336, 0.6831,  ..., 0.2506, 0.3447, 0.4566],
        ...,
        [1.2648, 1.1927, 1.1858,  ..., 1.7230, 1.5795, 1.4059],
        [0.5581, 0.6336, 0.6831,  ..., 0.2506, 0.3447, 0.4566],
        [0.8227, 1.0090, 1.1266,  ..., 0.4866, 0.5095, 0.6336]],
       dtype=torch.float64)

In [6]:
def landscape_minimas(landscape, filter = True):

    dims = len(landscape.shape)
    res = landscape.shape[0]

    minimas = torch.full(landscape.shape, True)

    for dim in range(dims):
        less_right = landscape <= torch.roll(landscape, 1 , dim)
        less_left = landscape <= torch.roll(landscape, -1 , dim)

        axis_minimas = torch.logical_and(less_right, less_left)
        minimas = torch.logical_and(minimas, axis_minimas)

    if filter:
        global_minimas = torch.isclose(landscape, torch.min(landscape))
        minimas = torch.logical_and(minimas, global_minimas)

    min_energy = landscape[minimas]

    points = torch.stack(torch.where(minimas)).T * (2*np.pi) / res
    gammas, betas = torch.hsplit(points, 2)

    return gammas, betas, min_energy

In [7]:
gammas, betas, min_energies = landscape_minimas(energy_scape)

gammas = gammas.requires_grad_()
betas = betas.requires_grad_()

print(f"{len(min_energies)} sample points")

256 sample points


In [8]:
optimizer = torch.optim.Adam([gammas, betas], lr=1e-3)

grad_mag = np.infty

count = 0
while grad_mag > 1e-4 and count < 10_000:
    optimizer.zero_grad()
    energy = torch_qaoa.energy(gammas, betas)
    loss = torch.mean(energy)
    
    loss.backward()
    optimizer.step()

    grad_mag = max(torch.max(gammas.grad.abs()), torch.max(betas.grad.abs()))

    print(f"{count}: loss = {loss}, grad = {grad_mag}", end="\t\t\r")

    count += 1

In [9]:
torch.min(energy)

tensor(-2.0947, dtype=torch.float64, grad_fn=<MinBackward1>)

In [10]:
torch_qaoa.energy(gammas, betas)

tensor([-2.0938, -2.0938, -2.0938, -2.0938, -2.0947, -2.0947, -2.0947, -2.0947,
        -2.0938, -2.0938, -2.0938, -2.0938, -2.0947, -2.0947, -2.0947, -2.0947,
        -2.0938, -2.0938, -2.0938, -2.0938, -2.0947, -2.0947, -2.0947, -2.0947,
        -2.0938, -2.0938, -2.0938, -2.0938, -2.0947, -2.0947, -2.0947, -2.0947,
        -2.0938, -2.0938, -2.0938, -2.0938, -2.0947, -2.0947, -2.0947, -2.0947,
        -2.0938, -2.0938, -2.0938, -2.0938, -2.0947, -2.0947, -2.0947, -2.0947,
        -2.0938, -2.0938, -2.0938, -2.0938, -2.0947, -2.0947, -2.0947, -2.0947,
        -2.0938, -2.0938, -2.0938, -2.0938, -2.0947, -2.0947, -2.0947, -2.0947,
        -2.0938, -2.0938, -2.0938, -2.0938, -2.0947, -2.0947, -2.0947, -2.0947,
        -2.0938, -2.0938, -2.0938, -2.0938, -2.0947, -2.0947, -2.0947, -2.0947,
        -2.0938, -2.0938, -2.0938, -2.0938, -2.0947, -2.0947, -2.0947, -2.0947,
        -2.0938, -2.0938, -2.0938, -2.0938, -2.0947, -2.0947, -2.0947, -2.0947,
        -2.0938, -2.0938, -2.0938, -2.09

In [103]:
torch.count_nonzero(torch.isclose(energy, torch.min(energy)))

tensor(32)

In [2]:
batches = 100
p = 3

gammas = torch.rand(batches, p) * 2 * np.pi
betas = torch.rand(batches, p) * 2 * np.pi

gammas = gammas.requires_grad_()
betas = betas.requires_grad_()

In [120]:
X = torch.tensor([[0, 1], [1, 0]], dtype=torch.complex128)
Y = torch.tensor([[0, -1j], [1j, 0]], dtype=torch.complex128)
Z = torch.tensor([[1, 0], [0, -1]], dtype=torch.complex128)
I = torch.tensor([[1, 0], [0, 1]], dtype=torch.complex128)

X_eigvecs = torch.tensor([[1, 1], [1, -1]], dtype=torch.complex128) / np.sqrt(2)
Y_eigvecs = torch.tensor([[1, 1], [1j, -1j]], dtype=torch.complex128) / np.sqrt(2)
Z_eigvecs = torch.tensor([[1, 0], [0, 1]], dtype=torch.complex128) / np.sqrt(2)


In [121]:
eigvals, eigvecs = torch.linalg.eigh(torch.kron(X, I))

eigvals

tensor([-1., -1.,  1.,  1.], dtype=torch.float64)

In [122]:
eigvecs

tensor([[-0.7071-0.j, -0.0000-0.j,  0.0000+0.j, -0.7071+0.j],
        [ 0.0000+0.j,  0.7071+0.j,  0.7071+0.j,  0.0000+0.j],
        [ 0.7071+0.j,  0.0000+0.j,  0.0000+0.j, -0.7071+0.j],
        [-0.0000-0.j, -0.7071-0.j,  0.7071+0.j,  0.0000+0.j]],
       dtype=torch.complex128)

In [123]:
torch.kron(X_eigvecs, X_eigvecs)

tensor([[ 0.5000+0.j,  0.5000+0.j,  0.5000+0.j,  0.5000+0.j],
        [ 0.5000+0.j, -0.5000+0.j,  0.5000+0.j, -0.5000+0.j],
        [ 0.5000+0.j,  0.5000+0.j, -0.5000+0.j, -0.5000+0.j],
        [ 0.5000+0.j, -0.5000+0.j, -0.5000+0.j,  0.5000-0.j]],
       dtype=torch.complex128)

In [68]:
def exp_QAOA(gammas, betas, driver_coupling, mixer_coupling):
    driver_eigvals, driver_eigvecs = coupling_eig(driver_coupling)
    mixer_eigvals, mixer_eigvecs = coupling_eig(mixer_coupling)

    param_shape = gammas.shape
    if len(param_shape) == 1:
        batches = 1
        p = param_shape[0]
    elif len(gammas.shape) == 2:
        batches, p = param_shape
    else:
        raise RuntimeError("gammas and betas must be 1 or 2 dimensional")

    # (2, batches, p)
    parameters = torch.stack((gammas, betas))

    # (2, N)
    eigvals = torch.stack([driver_eigvals, mixer_eigvals])

    # (2, N, N)
    eigvecs = torch.stack([driver_eigvecs, mixer_eigvecs])
    adjoint_eigvecs = torch.stack([driver_eigvecs.adjoint(), mixer_eigvecs.adjoint()])

    # (batches, p, 2, N) -> (2, batches, p, N)
    eigvals = eigvals.unsqueeze(0).unsqueeze(0).repeat(batches, p, 1, 1).permute(2, 0, 1, 3)

    # multiply by coefficients (2, batches, p), (2, batches, p, N) -> (2, batches, p, N)
    eigvals = torch.einsum('cbp,cbpn->cbpn', parameters * -1j, eigvals)

    # exponentiate
    eigvals = torch.exp(eigvals)

    # recombine (2, batches, p, N), (2, N, N) -> (2, batches, p, N, N)
    eigvals = torch.einsum('cbpi,cij->cbpij', eigvals, adjoint_eigvecs)

    # recombine (2, N, N), (2, batches, p, N, N) -> (2, batches, p, N, N) -> (p, 2, batches, N, N)
    eigvals = torch.einsum('cij, cbpjk->cbpik', eigvecs, eigvals)

    return eigvals.permute(2, 0, 1, 3, 4)

def parameter_eigs_exp(parameters, eigvals, eigvecs):

    N = len(eigvals)

    output_shape = parameters.shape + torch.Size([N, N])

    parameters = parameters.flatten()
    
    # (p), (N) -> (p, N)
    eig_exps = torch.exp(torch.einsum('p,N->pN', parameters, eigvals))

    # recombine (p, N), (N, N) -> (p, N, N)
    eig_exps = torch.einsum('pi,ij->pij', eig_exps, eigvecs.adjoint())
    eig_exps = torch.einsum('ij,pjk->pik', eigvecs, eig_exps)

    return eig_exps.reshape(output_shape)

In [69]:
res = 100
p = 1
s = torch.linspace(0, 2 * np.pi, res + 1)[:-1]
coords = torch.meshgrid(*[s] * (2 * p))
coords = torch.stack([coord.flatten() for coord in coords]).T
gammas, betas = torch.hsplit(coords, 2)

In [74]:
scape = exp_QAOA(gammas, betas, torch_qaoa.driver_coupling, torch_qaoa.mixer_coupling)
scape.shape

torch.Size([1, 2, 10000, 16, 16])

In [71]:
driver_eigvals, driver_eigvecs = coupling_eig(torch_qaoa.driver_coupling)
mixer_eigvals, mixer_eigvecs = coupling_eig(torch_qaoa.mixer_coupling)

scape_driver_ = parameter_eigs_exp(gammas * -1j, driver_eigvals, driver_eigvecs)
scape_mixer_ = parameter_eigs_exp(betas * -1j, mixer_eigvals, mixer_eigvecs)

In [75]:
scape_ = torch.stack([scape_driver_, scape_mixer_], 1)
scape_ = scape_.permute(2, 1, 0, 3, 4)
scape_.shape

torch.Size([1, 2, 10000, 16, 16])

In [76]:
(scape - scape_) ** 2

tensor([[[[[0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
           ...,
           [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j]],

          [[0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
           ...,
           [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j]],

          [[0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j